The Stillwater WWTP aeration tank parameters

There are 4 same tanks，in each tank：

L is the depth of the aeration tank, 57.0m.  
W is the depth of the aeration tank, 19.5m.  
H is the depth of the aeration tank, 8.0m. (Assume)  
Q is the inflow sewage flowrate, m3/s, 0.0525 $m^3$/s (Based on 2016 data)  
Qr is the return sludge sewage flowrate, 0.0400 $m^3$/s (Based on 2016 data)

In [16]:
'''
1.Hydraulics

Note：
1. If the calculated number of stages is greater than 12, set the N to 12 for modelling 
purposes (plug-flow reactor).
'''
L = 57
W = 10.5
H = 8
Q = 0.0525
Qr = 0.04

# The number of stages 
N = 7.4*(Q+Qr)*L/W/H;
if N<=1:
    N=1
if N>12:
    N=12;
N = round(N)
print ('The number of stages is:')
print (N)

# Hydraulic retention time, h
HRT=L*W*H/Q/3600;
print ('Hydraulic retention time (h) is:')
print (HRT)

# Tank volume, m3
Vol = L*W*H;
print ('The aeration tank volume (m3) is:')
print (Vol)

# the volume for each stage (m3) is:'), m3
Vi = Vol/N

The number of stages is:
1
Hydraulic retention time (h) is:
25.333333333333332
The aeration tank volume (m3) is:
4788.0


temperature is 15.3 Celsius  
SBOD is the soluable BOD concentration in the active sludge tank, 2.19 mg/l  
NO3 is the nitrate concentration in the active sludge tank, mg/l (ignoreed in areobic tank)  
NH3 is the ammonia concentration in the active sludge tank, 0.1 mg/l  
DO is the dissolved oxygen concentration in the active sludge tank, 9.2 mg/l (from the effluent, not accurate)

In [17]:
'''
2.Kinetic parameters

Note:
1. 
Saturation coefficient, BOD 
Saturation coefficient,O2 (BOD) 
Saturation coefficient, O2 (NH3) 
Saturation coefficient (NH3) 

These are the half-rate saturation constants in the Monod growth rate equations for 
heterotrophs (growth on BOD and oxygen) and autotrophs (growth on ammonia and oxygen).

2. The model allows the use of nitrate as an oxidising agent when this would favour bacterial 
growth. Only heterotrophic bacteria are capable of utilising nitrate. The decision as to which 
to use is made by calculating the growth rate on oxygen and nitrate and selecting whichever is 
the larger of the two. 

3. 
Heterotroph yield (mg/mgBOD) 
Autotroph yield (mg/mgNH3-N) 
Heterotroph yield (mg/mgNO3-N)

The yields are the sludge production per unit of BOD or ammonia removed. Under anoxic conditions 
the sludge yield is usually less than under aerobic.

4.
Heterotroph loss of viability fraction 
Heterotroph limiting viability 
Autotroph loss of viability fraction 
Autotroph limiting viability

The loss of viability fraction is the fraction of the maximum growth rate below which the 
bacteria are assumed to lose viability. By default this is 10% of the maximum growth rate. 
The limiting viability sets the maximum rate of loss of viability that the bacteria will undergo. 

'''
Temp = 15.3
# Tref is the reference temperature （15 Celsius） 
Tempref = 15;
#Heterotroph reference growth rate, 1/h
muHref=0.3
#Heterotroph growth rate temperature coefficient, 1/C
muHcoef=0.069
#Autotroph reference growth rate, 1/h
muAref=0.15
#Autotroph growth rate tempreture coefficient, 1/C
muAcoef=0.0729

import numpy
# Heterotroph max growth rate depends on tempreture, 1/h
muHtemp = muHref*numpy.exp(muHcoef*(Tempref-Temp));
# Autotroph max growth rate depends on tempreture, 1/h
muAtemp = muAref*numpy.exp(muAcoef*(Tempref-Temp));
#Saturation coefficient BOD, mg/l
KBOD=20
#Saturation coefficient,O2 (BOD), mg/l
KBODO=0.1
#Saturation coefficient, O2 (NH3), mg/l 
KNH3O=1
#Saturation coefficient (NH3), mg/l 
KNH3=0.5
#Saturation coefficient (NO3), mg/l
KNO3=10
#Heterotrophic enzyme max activity 
PsiHmax=0.1
#Autotrophic enzyme max activity
PsiAmax=0.5

SBOD = 2.19
NO3 = 0
NH3 = 0.1
DO= 9.2

# Heterotroph growth rate, 1/h
muaeroH = muHtemp*(SBOD/(KBOD+SBOD))*(DO/(KBODO+DO));
muanaeroH = muHtemp*(SBOD/(KBOD+SBOD))*(NO3/(KNO3+NO3));
if muaeroH>=muanaeroH:
    muH = muaeroH;
    YH=1.1 # Heterotroph yield (mg/mgBOD)
    PsiH = PsiHmax*(SBOD/(KBOD+SBOD))*(DO/(KBODO+DO)); #Heterotrophic enzyme activity 
    print ('The growth of heterotrophic bacteria is depending on oxygen. The growth rate (1/h) is:')
    print (muH)
    print ('Heterotroph aeorbic yield rate is:')
    print (YH)
    print ('Heterotrophic enzyme activity is:')
    print (PsiH)
if muaeroH<muanaeroH:
    muH = muanaeroH;
    YH=0.66
    PsiH = PsiHmax*(SBOD/(KBOD+SBOD))*(NO3/(KNO3+NO3)); #Heterotrophic enzyme activity 
    print ('The growth of heterotrophic bacteria is depending on nitrate. The growth rate (1/h) is:')
    print (muH)
    print ('Heterotroph anaerobic yield rate is:')
    print (YH)
    print ('Heterotrophic enzyme activity is:')
    print (PsiH)
    
# Autotroph growth rate, 1/h
muA = muAtemp*(NH3/(KNH3+NH3))*(DO/(KNH3O+DO));

# Autotroph yield (mg/mgNH3-N) 
YA=0.05

# Autotroph enzyme activity 
PsiA = PsiAmax*(NH3/(KNH3+NH3))*(DO/(KNH3O+DO)); 
print ('The growth of autotroph bacteria (1/h) is:')
print (muA)
print ('Autotroph yield rate is:')
print (YA)
print ('Autotroph enzyme activity is:')
print (PsiA)


#Heterotroph loss of viability fraction 
fH = 0.1
#Heterotroph limiting viability 
phiHmin = 0.95
#Autotroph loss of viability fraction 
fA = 0.1
#Autotroph limiting viability
phiAmin = 0.15

#Heterotroph loss of viability
phiH=muH*max(phiHmin,(1-muH/(fH*muHtemp)))
#Autotroph loss of viability
phiA=muA*max(phiAmin,(1-muA/(fA*muAtemp)))
print ('The heterotroph loss of viability is:')
print (phiH)
print ('The autotroph loss of viability is:')
print (phiA)

#Heterotroph reference death rate, 1/h
KDHref=0.005
#Heterotroph death rate temperature coefficient, 1/C
KDHcoef=0
#Autotroph reference death rate, 1/h
KDAref=0.005
#Autotroph death rate tempreture coefficient, 1/C
KDAcoef=0

#Heterotroph death rate depends on tempreture, 1/h
KDH=KDHref*numpy.exp(KDHcoef*(Tempref-Temp));
#Heterotroph death rate depends on tempreture, 1/h
KDA=KDAref*numpy.exp(KDAcoef*(Tempref-Temp));
print ('The heterotroph death rate (1/h) is:')
print (KDH)
print ('The autotroph death rate (1/h) is:')
print (KDA)

#Maintenance rate, 1/h
Mo = 0.000004

#Particulate BOD hydrolysis reference rate, 1/h
Rhydref = 0.125
#Particulate BOD hydrolysis temperature coefficient, 1/C
Rhydcoef = 0
#Hydrolysis saturation constant
Kx = 0.03

#Particulate BOD hydrolysis rate, 1/h:
Rhyd = Rhydref*numpy.exp(Rhydcoef*(Tempref-Temp));

#Oxygen required to oxidise unit mass of NH3-N
YONH3=2*32/17

The growth of heterotrophic bacteria is depending on oxygen. The growth rate (1/h) is:
0.028689504684124446
Heterotroph aeorbic yield rate is:
1.1
Heterotrophic enzyme activity is:
0.009763188881943333
The growth of autotroph bacteria (1/h) is:
0.022061226014323393
Autotroph yield rate is:
0.05
Autotroph enzyme activity is:
0.07516339869281047
The heterotroph loss of viability is:
0.02725502944991822
The autotroph loss of viability is:
0.003309183902148509
The heterotroph death rate (1/h) is:
0.005
The autotroph death rate (1/h) is:
0.005


The typical value for steady state calculation  

SBODin is the soluable BOD concentration in inflow, 60 mg/l (Assume)  
NH3in is the soluable BOD concentration in inflow, 24.63 mg/l (Assume)  
XBODin is the partculate BOD concentration  in inflow, 40 mg/l (Assume)  
DOin is the dissolved oxygen concentration  in inflow, 10 mg/l (Assumed by oxygen saturation @ 15 C)    
XTin is mixed liquor suspended solids (MLSS) in inflow, 53.9 mg/l  
XBOD is the partculate BOD concentration in tank, 2 mg/l (Assume)  
XT is mixed liquor suspended solids (MLSS) in tank, 1361.4 mg/l  

XVSSin is mixed liquor volatile suspended solids  (MLVSS) in inflow, 47.31 mg/l  
XVSS is mixed liquor volatile suspended solids  (MLVSS) in tank, 1131.23 mg/l  
XVSSr is mixed liquor volatile suspended solids  (MLVSS) in return activited sludge, 1879.23 mg/l  

In [18]:
'''
3. Concentrations of bacterials calculation at steady state calculation

Assume the concentrations of bacterials has a proportional relationship with MLVSS
'''  
SBODin = 60
NH3in = 24.63
XBODin = 40
DOin = 10
XTin = 53.9
XBOD =2
XT = 1361.4
XVSSin = 47.31 
XVSS = 1131.23
XVSSr = 1879.23

#XAV, Viable autotrophs, mg/l
#XANV, Nonviable autotrophs, mg/l
#XHV, Viable heterotrophs, mg/l
#XHNV, Nonviable heterotrophs, mg/l

from scipy.optimize import fsolve

def func(i):
     XAV, XANV, XHV, XHNV = i[0], i[1], i[2], i[3]
     return [(Q*XVSSin+Qr*XVSSr-(Q+Qr)*XVSS)/(Vol*XVSS)*3600-KDH*XHNV+phiH*XHV,
             (Q*XVSSin+Qr*XVSSr-(Q+Qr)*XVSS)/(Vol*XVSS)*3600-KDA*XANV+phiA*XAV,
             (Q+Qr)*(SBODin-SBOD)/Vol*3600-muH*XHV/YH-PsiH*XHNV+Rhyd*XBOD*XHV/(Kx*XT+XBOD),
             (Q+Qr)*(NH3in-NH3)/Vol*3600-muA*XAV/YA-PsiA*XANV
            ]             
r = fsolve(func,[1, 0, 100, 0])
print ('The concentration (mg/L) of Viable autotrophs, Nonviable autotrophs, Viable heterotrophs, Nonviable heterotrophs is:')
print (r)



The concentration (mg/L) of Viable autotrophs, Nonviable autotrophs, Viable heterotrophs, Nonviable heterotrophs is:
[  4.02399885  -0.92402859  55.20497989 297.3354115 ]


In [20]:
'''
4. Oxygen requirement at steady state

'''  
XAV = 4.024
XANV = 0
XHV = 55.205
XHNV = 297.335

OUR = -((Q*DOin+Qr*DO-(Q+Qr)*DO)/Vol*3600-muH*XHV/YH-PsiH*XHNV-YONH3*(muA*XAV/YA+PsiA*XANV)-Mo*DO*XT/(KBODO+DO))
print ('The required oxygen uptake rate (mg/L-h) is')
print (OUR)

The required oxygen uptake rate (mg/L-h) is
11.000755899600458
